## HW 1: Haunted Places

### CSV to TSV Converter
This python script converts the CSV file to TSV format using pandas. The script handles error cases and maintains the original data structure while changing the delimiter.

In [1]:
import pandas as pd

file_name = "haunted_places"

def convert_csv_to_tsv(input_csv, output_tsv):
    try:
        # Read the CSV file
        df = pd.read_csv(input_csv)
        
        # Write to TSV file
        df.to_csv(output_tsv, sep='\t', index=False)
        print(f"Successfully converted {input_csv} to {output_tsv}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    # Replace these with your actual file names
    input_csv_file = f"Datasets/{file_name}.csv"
    output_tsv_file = f"Datasets/{file_name}.tsv"
    
    convert_csv_to_tsv(input_csv_file, output_tsv_file)

Successfully converted Datasets/haunted_places.csv to Datasets/haunted_places.tsv


In [2]:
df_1 = _dntk.execute_sql(
  'SELECT *\nFROM \'Datasets/haunted_places.tsv\'',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled'
)
df_1

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097
...,...,...,...,...,...,...,...,...,...,...
10987,Westminster,United States,at 12 midnight you can see a lady with two lit...,city hall,Colorado,CO,-105.048936,39.862610,-105.037205,39.836653
10988,Westminster,United States,Is haunted by the victims of a murder that hap...,Pillar of Fire,Colorado,CO,-105.032091,39.847237,-105.037205,39.836653
10989,Wheat Ridge,United States,The institution was for kids 18 years old and ...,Ridge Mental Institution,Colorado,CO,-105.063974,39.769726,-105.077206,39.766098
10990,Wheat Ridge,United States,Gymnasium - their have been reports of a litt...,Wheat Ridge Middle School,Colorado,CO,-105.103613,39.764055,-105.077206,39.766098


### Haunted Places Evidence Analyzer
The below script processes the TSV file containing haunted places data and includes additional evidence columns. The script analyzes descriptions to extract audio evidence, visual evidence, dates, witness counts, and time of day, then outputs an updated TSV file with these new columns.

In [3]:
import pandas as pd
import datefinder
from datetime import datetime
import number_parser

def add_evidence_columns(input_file_path, output_file_path, audio_keywords, visual_keywords, event_keywords, apparition_keywords, time_keywords, witness_keywords):

    try:
        df = pd.read_csv(input_file_path, sep='\t')

        # Create the 'audio_evidence' column
        df['audio_evidence'] = df['description'].apply(
            lambda description: isinstance(description, str) and any(keyword in description.lower() for keyword in audio_keywords)
        )

        # Create the 'visual_evidence' column
        df['visual_evidence'] = df['description'].apply(
            lambda description: isinstance(description, str) and any(keyword in description.lower() for keyword in visual_keywords)
        )

        # Create the 'haunted_places_date' column
        df['haunted_places_date'] = df['description'].apply(
            lambda description: next(datefinder.find_dates(description), datetime(2025, 1, 1)).strftime('%Y/%m/%d')
            if isinstance(description, str) and any(datefinder.find_dates(description))
            else datetime(2025, 1, 1).strftime('%Y/%m/%d')
        )

        # Create the 'haunted_places_witness_count' column
        df['haunted_places_witness_count'] = df['description'].apply(
            lambda description: parse_witness_count(description, witness_keywords) if isinstance(description, str) else 0
        )
        
        # Create the 'time_of_day' column
        df['time_of_day'] = df['description'].apply(
            lambda description: discern_time_of_day(description, time_keywords) if isinstance(description, str) else "Unknown"
        )

        # Create the 'apparition_type' column
        df['apparition_type'] = df['description'].apply(
            lambda description: discern_type(description, apparition_keywords) if isinstance(description, str) else "Unknown"
        )

        # Create the 'event_type' column
        df['event_type'] = df['description'].apply(
            lambda description: discern_type(description, event_keywords) if isinstance(description, str) else "Unknown"
        )

        # Save the updated DataFrame to a new TSV file
        df.to_csv(output_file_path, sep='\t', index=False)
        print(f"Successfully created {output_file_path} with 'audio_evidence', 'visual_evidence', 'haunted_places_date', 'haunted_places_witness_count', 'time_of_day', 'apparition_type', and 'event_type' columns.")

    except FileNotFoundError:
        print(f"Error: File not found at {input_file_path}")
    except pd.errors.EmptyDataError:
        print(f"Error: The file at {input_file_path} is empty.")
    except KeyError:
        print("Error: 'description' column not found in the file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def parse_witness_count(description, witness_keywords):

    try:
        description_lower = description.lower()
        
        # Check for any of the specified witness keywords in the description
        if any(keyword in description_lower for keyword in witness_keywords):
            words = description_lower.split()
            
            for i, word in enumerate(words):
                # If the word is a witness keyword
                if word in witness_keywords:
                    # Check if the next word is "by"
                    if i + 1 < len(words) and words[i + 1] == "by":
                        # Attempt to parse the number after "by"
                        try:
                            witness_count = number_parser.parse(words[i + 2])
                            return int(witness_count)
                        except (ValueError, IndexError):
                            pass  # Parsing failed, continue searching
                else:
                    try:
                        witness_count = number_parser.parse(word)
                        if isinstance(witness_count, int):
                            return int(witness_count)
                    except (ValueError, IndexError):
                        pass
        return 0  # No witness count found
    except Exception:
        return 0

def discern_time_of_day(description, time_keywords):
    description_lower = description.lower()
    for time_period, keywords in time_keywords.items():
        if any(keyword in description_lower for keyword in keywords):
            return time_period
    return "Unknown"

def discern_type(description, keywords_dict):
    description_lower = description.lower()
    
    for type_name, keywords in keywords_dict.items():
        if any(keyword in description_lower for keyword in keywords):
            return type_name
    return "Unknown"

# Example usage:
if __name__ == "__main__":
    input_file = 'Datasets/haunted_places.tsv'
    output_file = 'Datasets/haunted_places_evidence.tsv'
    audio_keywords_to_search = ['noises', 'sound', 'voices']  # We can add as many as needed
    visual_keywords_to_search = ['camera', 'pictures', "visual"]  # Feel free to add more.
    event_keywords_to_search = {
        "Murder": ["murder", "killed", "stabbed"],
        "Death": ["died", "drowned", "passed away"],
        "Supernatural Phenomenon": ["haunted", "ghostly", "paranormal"]
    }
    apparition_keywords_to_search = {
        "Ghost": ["ghost", "specter", "phantom"],
        "Orb": ["orb", "light ball"],
        "UFO": ["ufo", "unidentified flying object"],
        "UAP": ["uap", "unidentified aerial phenomena"]
    }
    time_keywords_to_search = {
        "Morning": ["morning", "sunrise"],
        "Evening": ["evening", "night", "sunset", "dark"],
        "Dusk": ["dusk"]
    }
    witness_keywords_to_search = ["witness", "witnessed", "seen", "saw"]
    add_evidence_columns(input_file, output_file, audio_keywords_to_search, visual_keywords_to_search, event_keywords_to_search, apparition_keywords_to_search, time_keywords_to_search, witness_keywords_to_search)

/etc/timezone is deprecated on Debian, and no longer reliable. Ignoring.
Successfully created Datasets/haunted_places_evidence.tsv with 'audio_evidence', 'visual_evidence', 'haunted_places_date', 'haunted_places_witness_count', 'time_of_day', 'apparition_type', and 'event_type' columns.


In [4]:
df_2 = _dntk.execute_sql(
  'SELECT *\nFROM \'Datasets/haunted_places_evidence.tsv\'',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled'
)
df_2

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude,audio_evidence,visual_evidence,haunted_places_date,haunted_places_witness_count,time_of_day,apparition_type,event_type
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,False,False,2025-02-20,0,Evening,Ghost,Murder
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,False,False,2025-02-19,0,Unknown,Unknown,Murder
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,False,False,2025-01-01,0,Evening,Unknown,Murder
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,False,False,1970-02-19,0,Morning,Unknown,Unknown
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,False,False,2025-01-01,0,Unknown,Unknown,Death
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10987,Westminster,United States,at 12 midnight you can see a lady with two lit...,city hall,Colorado,CO,-105.048936,39.862610,-105.037205,39.836653,False,False,2025-12-19,0,Evening,Unknown,Murder
10988,Westminster,United States,Is haunted by the victims of a murder that hap...,Pillar of Fire,Colorado,CO,-105.032091,39.847237,-105.037205,39.836653,False,False,2025-01-01,0,Unknown,Ghost,Murder
10989,Wheat Ridge,United States,The institution was for kids 18 years old and ...,Ridge Mental Institution,Colorado,CO,-105.063974,39.769726,-105.077206,39.766098,True,True,2007-02-19,0,Unknown,Orb,Supernatural Phenomenon
10990,Wheat Ridge,United States,Gymnasium - their have been reports of a litt...,Wheat Ridge Middle School,Colorado,CO,-105.103613,39.764055,-105.077206,39.766098,False,False,2025-01-01,0,Morning,Ghost,Death


In [5]:
df_3 = _dntk.execute_sql(
  'SELECT \n    COUNT(CASE WHEN audio_evidence = TRUE THEN 1 END) AS audio_true_count,\n    COUNT(CASE WHEN audio_evidence = FALSE THEN 1 END) AS audio_false_count,\n    COUNT(CASE WHEN visual_evidence = TRUE THEN 1 END) AS visual_true_count,\n    COUNT(CASE WHEN visual_evidence = FALSE THEN 1 END) AS visual_false_count,\n    COUNT(CASE WHEN haunted_places_witness_count > 0 THEN 1 END) AS num_of_lines_with_more_then_0,\n    COUNT(CASE WHEN haunted_places_date != \'2025-01-01 00:00:00\' THEN 1 END) AS haunted_date_not_blank,\n    COUNT(CASE WHEN time_of_day != \'Unknown\' THEN 1 END) AS time_of_day_not_unknown,\n    COUNT(CASE WHEN event_type != \'Unknown\' THEN 1 END) AS event_type_not_unknown\nFROM \'Datasets/haunted_places_evidence.tsv\'',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled'
)
df_3

,audio_true_count,audio_false_count,visual_true_count,visual_false_count,num_of_lines_with_more_then_0,haunted_date_not_blank,time_of_day_not_unknown,event_type_not_unknown
0,2096,8896,273,10719,7,10992,3943,4434


### Word Count

In [6]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet as wn
def get_word_category(word, pos_tag):
    # Convert POS tag to WordNet POS tag format
    pos_map = {
        'NN': wn.NOUN, 'NNS': wn.NOUN, 'NNP': wn.NOUN, 'NNPS': wn.NOUN,
        'VB': wn.VERB, 'VBD': wn.VERB, 'VBG': wn.VERB, 'VBN': wn.VERB, 
        'VBP': wn.VERB, 'VBZ': wn.VERB,
        'JJ': wn.ADJ, 'JJR': wn.ADJ, 'JJS': wn.ADJ,
        'RB': wn.ADV, 'RBR': wn.ADV, 'RBS': wn.ADV
    }
    
    wn_pos = pos_map.get(pos_tag, None)
    if not wn_pos:
        return 'other' 
    
    # Get synsets (sets of synonyms) for the word
    synsets = wn.synsets(word, pos=wn_pos)
    if not synsets:
        return 'other'
    
    # Get the most common synset
    synset = synsets[0]
    
    # Get all hypernyms 
    hypernyms = []
    for hypernym_path in synset.hypernym_paths():
        hypernyms.extend(hypernym_path)
    
    # Define categories based on WordNet hierarchy
    categories = {
        'perception_verb': {'perceive.v.01', 'sense.v.01', 'hear.v.01', 'see.v.01'},
        'motion_verb': {'move.v.01', 'travel.v.01', 'go.v.01'},
        'communication_verb': {'communicate.v.01', 'tell.v.01', 'say.v.01'},
        'emotion_verb': {'feel.v.01', 'emotion.n.01'},
        'building': {'building.n.01', 'structure.n.01'},
        'person': {'person.n.01', 'human.n.01'},
        'location': {'location.n.01', 'place.n.01'},
        'supernatural': {
            'supernatural_being.n.01', 'spirit.n.01', 'ghost.n.01', 
            'apparition.n.01', 'phantom.n.01', 'specter.n.01', 
            'supernatural.n.01', 'demon.n.01', 'angel.n.01'
        }
    }
    
    # Check if the word itself is in any target categories
    synsets = wn.synsets(word, pos=wn_pos)
    if synsets:
        for synset in synsets:
            for category, marker_synsets in categories.items():
                if synset.name() in marker_synsets:
                    return category
    
    # Check hypernyms if no direct match
    if synsets:
        synset = synsets[0]
        hypernyms = []
        for hypernym_path in synset.hypernym_paths():
            hypernyms.extend(hypernym_path)
        
        for category, marker_synsets in categories.items():
            if any(h.name() in marker_synsets for h in hypernyms):
                return category
    return 'other'

def count_word_occurrences(filename):
    df = pd.read_csv(filename)
    
    # Concatenate all rows in the column into a single string
    all_text = ' '.join(df["description"].dropna().astype(str)).lower()
    
    # Tokenize the text into words
    words = all_text.split()

    # Remove "noise" stop words and non-alphanumeric words
    stop_words = set(stopwords.words('english'))
    
    # Create a dictionary to count word occurrences
    word_counts = {}
    for word in words:
        if not word.isalnum() or word.lower() in stop_words:
            continue
        elif word.isalnum():
            word_counts[word] = word_counts.get(word, 0) + 1
    
    # Convert the dictionary into a DataFrame
    word_counts_df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Count'])

    # Add POS tags
    pos_tags = {word: pos_tag([word])[0][1] for word in word_counts_df['Word']}
    word_counts_df['POS'] = word_counts_df['Word'].map(pos_tags)
    
    # Add semantic categories using WordNet
    word_counts_df['Category'] = word_counts_df.apply(
        lambda row: get_word_category(row['Word'].lower(), row['POS']), axis=1
    )
    
    # Sort by count in descending order
    word_counts_df = word_counts_df.sort_values(by='Count', ascending=False).reset_index(drop=True)
    word_counts_df.to_csv("word_counts.csv", index=False)
    return word_counts_df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Dataset Collection and MIME Types

### Alcohol Consumption Data By State

**Source:** National Center for Drug Abuse Statistics (NCDAS)
**URL:** https://drugabusestatistics.org/alcohol-abuse-statistics/
**MIME Type:** application/html
**Features:**
- Binge Drinking Rate
- Annual Alcohol Deaths
- Binge Times Monthly

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Site URL
url = "https://drugabusestatistics.org/alcohol-abuse-statistics/"

# Fetch webpage content
response = requests.get(url)
response.raise_for_status()

# Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Extract all text content
text_content = soup.get_text(separator="\n")

# Initialize state_data list
state_data = []

# Function to extract data for a given state
def extract_state_data(state_name, text_content):
    # Extract binge drinking rate
    binge_pattern = re.search(r"(\d+\.\d+|\d+)%\s*of\s*.*?(binge drink|excessive drinking)", text_content, re.IGNORECASE)

    # Extract annual deaths attributable to excessive alcohol use
    deaths_pattern = re.search(r"(An average of\s*)?([\d,]+)\s*(annual )?(deaths?|fatalities).*?attributable to excessive alcohol use", text_content, re.IGNORECASE)

    # Extract number of times adults binge drink monthly
    binge_times_pattern = re.search(r"(Binge drinking adults|Adults who binge drink).*?(\d+\.\d+|\d+)\s*times? (per month|monthly)", text_content, re.IGNORECASE)

    # Assign extracted values, or "N/A" if not found
    binge_rate = binge_pattern.group(1) if binge_pattern else "N/A"
    deaths_alcohol = deaths_pattern.group(2) if deaths_pattern else "N/A"
    binge_times_monthly = binge_times_pattern.group(2) if binge_times_pattern else "N/A"

    return [state_name, 2023, binge_rate, deaths_alcohol, binge_times_monthly]

# Extract data for each state
state_sections = re.split(r'((?:[A-Z][a-z]+(?:\s[A-Z][a-z]+)*) Alcohol Abuse Statistics)', text_content)

# Iterate through state sections and extract data
for i in range(1, len(state_sections), 2):
    state_name = state_sections[i].replace(" Alcohol Abuse Statistics", "").strip()
    state_text = state_sections[i + 1]
    
    # Special handling for District of Columbia
    if state_name == "Columbia":
        state_name = "District of Columbia"
    
    state_data.append(extract_state_data(state_name, state_text))

# Special handling for District of Columbia if not found in the main loop
dc_names = ["District of Columbia", "D.C.", "Washington D.C."]
dc_data = None
for dc_name in dc_names:
    dc_data = extract_state_data(dc_name, text_content)
    if dc_data[2] != "N/A" or dc_data[3] != "N/A" or dc_data[4] != "N/A":
        # Check if District of Columbia is already in state_data
        if not any(data[0] == "District of Columbia" for data in state_data):
            state_data.append(dc_data)
        break

# If D.C. data is still not found, add a placeholder
if dc_data is None or (dc_data[2] == "N/A" and dc_data[3] == "N/A" and dc_data[4] == "N/A"):
    if not any(data[0] == "District of Columbia" for data in state_data):
        state_data.append(["District of Columbia", 2023, "N/A", "N/A", "N/A"])

# Convert to DataFrame
df_alcohol_stats = pd.DataFrame(
    state_data, columns=["State", "Year", "Binge_Drinking_Rate", "Annual_Alcohol_Deaths", "Binge_Times_Monthly"]
)

# Remove any empty rows or artifacts
df_alcohol_stats = df_alcohol_stats[df_alcohol_stats["State"].notna()]

# Display first few rows
print(df_alcohol_stats.head(10))

# Save DataFrame to CSV
df_alcohol_stats.to_csv("Exports/alcohol_data/alcohol_abuse_by_state.csv", index=False)

print("Data extraction complete! Saved as 'alcohol_abuse_by_state.csv'")


                  State  Year Binge_Drinking_Rate Annual_Alcohol_Deaths  \
0               Alabama  2023                13.6                 2,208   
1                Alaska  2023                16.0                   433   
2               Arizona  2023                16.7                 3,670   
3              Arkansas  2023                16.0                 1,407   
4            California  2023                16.6                15,443   
5              Colorado  2023                19.8                 2,623   
6           Connecticut  2023                18.7                 1,426   
7              Delaware  2023                18.3                   466   
8  District of Columbia  2023                22.5                   392   
9               Florida  2023                17.5                10,655   

  Binge_Times_Monthly  
0                 1.7  
1                 1.9  
2                 1.5  
3                 1.9  
4                 1.6  
5                 1.6  
6     

### Daylight Duration Data By State

**Source:** U.S. Naval Observatory
**URL:** https://aa.usno.navy.mil/data/Dur_OneYear
**MIME Type:** text/html
**Features:**
- Average Daylight Hours Per Year
- Winter Solstice Daylight Hours
- Summer Solstice Daylight Hours

In [41]:
import aiohttp
import asyncio
import nest_asyncio
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import urllib.parse

nest_asyncio.apply()

async def fetch_daylight_data(session, latitude, longitude, date):
    base_url = "https://aa.usno.navy.mil/calculated/rstt/oneday"
    params = {
        "date": date.strftime("%Y-%m-%d"),
        "lat": f"{latitude:.4f}",
        "lon": f"{longitude:.4f}",
        "label": "test",
        "tz": 0.00,
        "tz_sign": 1,
        "tz_label": "false",
        "dst": "false",
        "submit": "Get Data"
    }
    
    full_url = base_url + "?" + urllib.parse.urlencode(params)
    print(f"Scraping URL: {full_url}")

    try:
        async with session.get(full_url) as response:
            response.raise_for_status()
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')

            def find_table_with_rise_set(soup):
                tables = soup.find_all('table')
                for table in tables:
                    if 'Rise' in table.text and 'Set' in table.text:
                        return table
                return None

            table = find_table_with_rise_set(soup)
            if not table:
                print("Could not find results table containing 'Rise' and 'Set'")
                return None
            print("Successfully found results table")

            rows = table.find_all('tr')
            if len(rows) < 3:
                print("Could not find enough rows in the table (expected at least 3)")
                return None
            print("Successfully found enough rows in the table")

            sunrise_text = None
            sunset_text = None

            for row in rows:
                cells = row.find_all('td')
                if cells and len(cells) > 0 and cells[0].text.strip() == "Rise":
                    sunrise_text = cells[1].text.strip()
                if cells and len(cells) > 0 and cells[0].text.strip() == "Set":
                    sunset_text = cells[1].text.strip()

            if sunrise_text and sunset_text:
                print(f"Sunrise text: {sunrise_text}, Sunset text: {sunset_text}")
            else:
                print("Could not find Rise or Set times in the table")
                return None

            try:
                sunrise = datetime.strptime(sunrise_text, "%H:%M")
                sunset = datetime.strptime(sunset_text, "%H:%M")
                print("Successfully parsed sunrise and sunset times")

                daylight_duration = datetime.combine(date, sunset.time()) - datetime.combine(date, sunrise.time())
                daylight_hours = daylight_duration.total_seconds() / 3600
                print(f"Daylight duration: {daylight_hours:.2f} hours")
                print(f"Daylight duration (timedelta): {daylight_duration}")
                return daylight_hours

            except ValueError:
                print(f"Could not parse sunrise/sunset times: sunrise='{sunrise_text}', sunset='{sunset_text}'")
                return None

    except aiohttp.ClientError as e:
        print(f"Error scraping {base_url}: {e}")
        return None

async def main():
    print("Starting main function")
    haunted_places_file = "Datasets/haunted_places_evidence.tsv"
    try:
        haunted_df = pd.read_csv(haunted_places_file, sep='\t')
        print(f"Successfully loaded {haunted_places_file}")
    except FileNotFoundError:
        print(f"Error: {haunted_places_file} not found.")
        return

    haunted_df['latitude'] = pd.to_numeric(haunted_df['latitude'], errors='coerce')
    haunted_df['longitude'] = pd.to_numeric(haunted_df['longitude'], errors='coerce')
    print("Successfully converted latitude and longitude to numeric")

    haunted_df['average_daylight_hours'] = None
    today = datetime.now()

    async with aiohttp.ClientSession() as session:
        tasks = []
        for index, row in haunted_df.iterrows():
            latitude = row['latitude']
            longitude = row['longitude']

            if pd.isna(latitude) or pd.isna(longitude):
                print(f"Skipping row {index} due to invalid latitude or longitude")
                continue

            print(f"Scraping data for latitude: {latitude}, longitude: {longitude}")
            task = asyncio.ensure_future(fetch_daylight_data(session, latitude, longitude, today))
            tasks.append(task)

        daylight_hours = await asyncio.gather(*tasks)

    for index, hours in enumerate(daylight_hours):
        if hours is not None:
            haunted_df.loc[index, 'average_daylight_hours'] = hours
            row = haunted_df.iloc[index]
            latitude = row['latitude']
            longitude = row['longitude']
            print(f"Daylight data for {latitude}, {longitude}: {hours:.2f} hours")
        else:
            row = haunted_df.iloc[index]
            latitude = row['latitude']
            longitude = row['longitude']
            print(f"Could not retrieve daylight data for {latitude}, longitude: {longitude}")
            print('---------')

    output_file = "Exports/daylight_duration_data/haunted_places_evidence_daylight.tsv"
    haunted_df.to_csv(output_file, sep='\t', index=False)
    print(f"Successfully merged and saved data to {output_file}")

if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


Daylight duration (timedelta): 10:15:00
Successfully found results table
Successfully found enough rows in the table
Sunrise text: 05:01, Sunset text: 15:20
Successfully parsed sunrise and sunset times
Daylight duration: 10.32 hours
Daylight duration (timedelta): 10:19:00
Successfully found results table
Successfully found enough rows in the table
Sunrise text: 05:13, Sunset text: 15:29
Successfully parsed sunrise and sunset times
Daylight duration: 10.27 hours
Daylight duration (timedelta): 10:16:00
Successfully found results table
Successfully found enough rows in the table
Sunrise text: 05:13, Sunset text: 15:29
Successfully parsed sunrise and sunset times
Daylight duration: 10.27 hours
Daylight duration (timedelta): 10:16:00
Successfully found results table
Successfully found enough rows in the table
Sunrise text: 05:00, Sunset text: 15:20
Successfully parsed sunrise and sunset times
Daylight duration: 10.33 hours
Daylight duration (timedelta): 10:20:00
Successfully found results t

### Crime Rates By State

**Source:** FBI's Crime Data Explorer
**URL:** https://cde.ucr.cjis.gov/
**MIME Type:** application/json
**Features:**
- Violent Crime Rate Per 100K
- % of Violent Crimes Solved
- Homicide Incidents Per 100K

In [1]:
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

BASE_URL = "https://csg-state-violent-crime.netlify.app/state-viol-crime-{}"

STATE_ABBREVIATIONS = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California", "CO": "Colorado",
    "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana",
    "ME": "Maine", "MD": "Maryland", "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota",
    "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont", "VA": "Virginia", "WA": "Washington",
    "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming"
}

crime_data = []

for state_abbr, state_name in STATE_ABBREVIATIONS.items():
    print(f"Fetching crime data for {state_name} ({state_abbr})...")

    url = BASE_URL.format(state_abbr.lower())
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch data for {state_name}. Skipping...")
        continue

    soup = BeautifulSoup(response.text, "html.parser")

    try:
        violent_crime_text = soup.find(string=lambda t: t and "violent crime incidents per 100,000 residents were reported to police in " + state_name in t)
        crimes_solved_text = soup.find(string=lambda t: t and "percent of violent crimes reported to police in " + state_name + " were solved" in t)

        year = "N/A"
        violent_crime_rate = "N/A"
        percent_solved = "N/A"
        homicide_rate = "N/A"

        if violent_crime_text:
            year_match = re.search(r'In (\d{4}),', violent_crime_text)
            year = year_match.group(1) if year_match else "N/A"
            
            rate_match = re.search(r'In \d{4}, (\d+) violent crime incidents', violent_crime_text)
            violent_crime_rate = rate_match.group(1) if rate_match else "N/A"

        if crimes_solved_text:
            solved_match = re.search(r'In \d{4}, (\d+) percent of violent crimes', crimes_solved_text)
            percent_solved = solved_match.group(1) if solved_match else "N/A"

        # Extract homicide rate from the JSON data in the script tag
        script_tag = soup.find('script', {'data-for': lambda x: x and x.startswith('htmlwidget-')})
        if script_tag:
            json_text = script_tag.string
            data = json.loads(json_text)
            series_data = data["x"]["hc_opts"]["series"]
            state_data = next((s for s in series_data if s.get("name") == state_name), None)
            if state_data:
                data_2023 = next((d for d in state_data["data"] if d.get("year") == 2023), None)
                if data_2023:
                    homicide_rate = data_2023["incidents_reported_rate_total"]

        crime_data.append({
            "State": state_name,
            "Year": year,
            "Violent_Crime_Per_100k": violent_crime_rate,
            "Percent_of_Crimes_Solved": percent_solved,
            "Homicide_Incidents_Per_100k": homicide_rate
        })

        print(f"Successfully extracted data for {state_name}: Year = {year}, Violent Crime Rate = {violent_crime_rate}, Percent Solved = {percent_solved}, Homicide Rate = {homicide_rate}")

    except Exception as e:
        print(f"Could not extract data for {state_name}: {e}")

df_crime = pd.DataFrame(crime_data)
df_crime.to_csv("Exports/crime_rates_data/crime_data_by_state.csv", index=False)
print("Data extraction complete! Saved as 'crime_data_by_state.csv'.")
print(df_crime)

Fetching crime data for Alabama (AL)...
Successfully extracted data for Alabama: Year = 2023, Violent Crime Rate = 404, Percent Solved = 37, Homicide Rate = 403.9371490630851
Fetching crime data for Alaska (AK)...
Successfully extracted data for Alaska: Year = 2023, Violent Crime Rate = 726, Percent Solved = 54, Homicide Rate = 726.337117503811
Fetching crime data for Arizona (AZ)...
Successfully extracted data for Arizona: Year = 2023, Violent Crime Rate = 409, Percent Solved = 39, Homicide Rate = 408.8493279277611
Fetching crime data for Arkansas (AR)...
Successfully extracted data for Arkansas: Year = 2023, Violent Crime Rate = 620, Percent Solved = 43, Homicide Rate = 619.8716185116562
Fetching crime data for California (CA)...
Successfully extracted data for California: Year = 2023, Violent Crime Rate = 508, Percent Solved = 40, Homicide Rate = 508.2382114724801
Fetching crime data for Colorado (CO)...
Successfully extracted data for Colorado: Year = 2023, Violent Crime Rate = 474

### Air Pollution Data

**Source:** Regional Air Quality Data 
**URL1:** https://catalog.data.gov/dataset/regional-air-quality-data (metadata source is in JSON format)
**URL2:** EPA.gov - https://www.epa.gov/castnet/download-data (zip files are in excel CSV format)
**MIME Type:** application/json, text/csv, application/csv
**Reasoning:** Carbon monoxide poisoning causes hallucinations (and has been a common explanation for hauntings/supernatural activity). Might be able to do something similar with areas experiencing high levels of air pollution. Can replace this with a carbon monoxide poisoning dataset if we come across one.
**Features:**
1. Ozone (O₃) Concentration
 - **Description:** Represents the levels of ground-level ozone, a major air pollutant that affects human health and visibility.
- **Hypothesis:** High ozone levels can lead to eye irritation, headaches, and breathing difficulties, which may cause people to misinterpret their physical symptoms as supernatural encounters. Additionally, ozone is linked to electrical storms and atmospheric disturbances, which could contribute to eerie environments where ghost sightings are reported.
- **Additional Data Source:** EPA CASTNET Ozone Levels, which provide detailed data on regional ozone concentrations.
- **Query to use:** Are ghost sightings more frequent in locations with high ozone concentrations, suggesting that atmospheric conditions may influence perceived hauntings?

2. Sulfur Dioxide (SO₂) Concentration
- **Description:** Measures the concentration of sulfur dioxide in the air, which is commonly associated with industrial emissions and volcanic activity. It has a distinct, pungent smell and can cause respiratory discomfort and irritation.
- **Hypothesis:** High levels of SO₂ exposure can cause dizziness, headaches, and difficulty breathing, potentially contributing to eerie or otherworldly sensations experienced in haunted locations. It may also create a sulfuric odor, often linked in folklore to demonic or paranormal activity.
- **Additional Data Source:** EPA CASTNET SO₂ Levels, which provide historical sulfur dioxide measurements at different monitoring stations across the U.S.
- **Query to use:** Are haunted locations with stronger ghostly activity correlated with higher SO₂ levels, indicating possible environmental effects influencing supernatural experiences?
3. Particulate Matter Composition
- **Description:** Measures the concentration of airborne particulate matter, including sulfates, nitrates, and ammonium, which can reduce visibility and affect human health.
- **Hypothesis:** High levels of particulate pollution can lead to "visual distortions" like haze or fog, which may be mistaken for paranormal apparitions such as ghostly figures or floating orbs. Additionally, fine particulates can impact neurological function, potentially leading to hallucinations or altered perceptions in certain individuals.
- **Additional Data Source:** EPA CASTNET Particulate Matter (PM) Levels, which include sulfate, nitrate, and ammonium concentrations.
- **Query to use:** Do haunted locations with a history of spectral apparitions coincide with higher levels of particulate pollution, suggesting that air quality impacts the visibility and perception of supernatural events?

In [1]:
import os
import re
import zipfile
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib3

# Suppress SSL warning messages
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# EPA CASTNET Data Directory
EPA_DATA_URL = "https://gaftp.epa.gov/castnet/CASTNET_Outgoing/data/"
OUTPUT_CSV = "/work/Exports/air pollution data/air_pollution_data.csv"
DATA_FOLDER = "/work/Exports/air pollution data/castnet_data"
EXTRACT_FOLDER = "/work/Exports/air pollution data/extracted_data"

# Ensure directories exist
os.makedirs(DATA_FOLDER, exist_ok=True)
os.makedirs(EXTRACT_FOLDER, exist_ok=True)

def get_zip_links(base_url):
    """Web scrapes the FTP directory for ZIP file links."""
    response = requests.get(base_url, verify=False)  # SSL verification disabled
    if response.status_code != 200:
        print(f"Failed to access {base_url}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    links = [a["href"] for a in soup.find_all("a", href=True)]

    # Filter only relevant ZIP files (ozone, drydep, historic, etc.)
    zip_links = [urljoin(base_url, link) for link in links if re.search(r"(ozone|drydep|historic|wetchem).*\.zip$", link)]
    return zip_links

def download_zip_files(zip_links, save_folder):
    """Downloads ZIP files from the provided links."""
    for link in zip_links:
        filename = os.path.join(save_folder, os.path.basename(link))
        if os.path.exists(filename):
            print(f"Skipping {filename}, already downloaded.")
            continue

        response = requests.get(link, verify=False)  # SSL verification disabled
        if response.status_code == 200:
            with open(filename, "wb") as file:
                file.write(response.content)
            print(f"Downloaded: {filename}")
        else:
            print(f"Failed to download: {link}")

def extract_zip_files(zip_folder, extract_to):
    """Extracts all ZIP files in the given folder."""
    for file in os.listdir(zip_folder):
        if file.endswith(".zip"):
            file_path = os.path.join(zip_folder, file)
            try:
                with zipfile.ZipFile(file_path, "r") as zip_ref:
                    zip_ref.extractall(extract_to)
                print(f"Extracted: {file_path}")
            except zipfile.BadZipFile:
                print(f"Error extracting: {file_path}")

def process_air_quality_data(folder_path):
    """Processes extracted CSV files to extract relevant air pollution data."""
    all_data = []

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            try:
                print(f"Processing {file}...")

                # Read only a sample (first 10,000 rows) to prevent crashes
                df_sample = pd.read_csv(file_path, nrows=10000, low_memory=False, on_bad_lines='skip')

                # Convert column names to lowercase
                df_sample.columns = df_sample.columns.str.lower()

                # Detect alternative column names dynamically
                col_mapping = {
                    "site_id": ["site_id"],
                    "date_time": ["date_time"],
                    "year": ["year"],
                    "o3": ["ozone", "o3_concentration", "ozone_ppb"],
                    "so2": ["so2_conc", "sulfur_dioxide", "so2_ppb"],
                    "pm": ["pm_concentration", "pm25", "pm10"]
                }

                selected_cols = {}
                for key, alt_names in col_mapping.items():
                    for alt_name in alt_names:
                        if alt_name in df_sample.columns:
                            selected_cols[alt_name] = key
                            break  # Stop checking after finding the first match

                if selected_cols:
                    print(f"Using columns for {file}: {selected_cols}")

                    # Read full CSV with matched columns
                    df = pd.read_csv(file_path, usecols=selected_cols.keys(), low_memory=False, on_bad_lines='skip')

                    # Rename columns
                    df.rename(columns=selected_cols, inplace=True)

                    # Extract Year from DATE_TIME if it exists
                    if "date_time" in df.columns:
                        df["year"] = pd.to_datetime(df["date_time"], errors="coerce").dt.year
                        df.drop(columns=["date_time"], inplace=True)  # Drop original datetime column

                    all_data.append(df)
                else:
                    print(f"Skipping {file}: No relevant columns found.")

            except pd.errors.ParserError as e:
                print(f"Skipping {file}: CSV Parsing Error - {e}")
            except Exception as e:
                print(f"Error processing {file}: {e}")

    # Merge all extracted data
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_csv(OUTPUT_CSV, index=False)
        print(f"Processed data saved to {OUTPUT_CSV}")
    else:
        print("No valid data extracted.")

# Main Execution
if __name__ == "__main__":
    print("Scraping ZIP links...")
    zip_links = get_zip_links(EPA_DATA_URL)
    
    print(f"Found {len(zip_links)} relevant ZIP files. Downloading...")
    download_zip_files(zip_links, DATA_FOLDER)
    
    print("Extracting ZIP files...")
    extract_zip_files(DATA_FOLDER, EXTRACT_FOLDER)
    
    print("Processing extracted files...")
    process_air_quality_data(EXTRACT_FOLDER)

    print(f"Data extraction complete! Saved as {OUTPUT_CSV}")


Scraping ZIP links...
Found 56 relevant ZIP files. Downloading...
Skipping /work/Exports/air pollution data/castnet_data/drydep_week_web.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/historic_average.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/historic_average_week.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/ozone.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/ozone_1987.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/ozone_1988.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/ozone_1989.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/ozone_1990.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/ozone_1991.zip, already downloaded.
Skipping /work/Exports/air pollution data/castnet_data/ozone_1992.zip, already downloaded.
Skippin

The below script checks for column headers in all zip files.

In [9]:
import os
import pandas as pd

EXTRACTED_FOLDER = "/work/Exports/air pollution data/extracted_data"
OUTPUT_CSV = "/work/Exports/air pollution data/air_pollution_data.csv"

def process_air_quality_data(folder_path):
    """Processes extracted CSV files to extract relevant air pollution data."""
    all_data = []

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            try:
                print(f"Processing {file}...")

                # Read first 10 rows to detect columns
                df_sample = pd.read_csv(file_path, nrows=10, low_memory=False, on_bad_lines='skip')

                # Print detected column names
                print(f"Columns found in {file}: {list(df_sample.columns)}")

                # Convert column names to lowercase
                df_sample.columns = df_sample.columns.str.lower()

                # Detect alternative column names dynamically
                col_mapping = {
                    "state": ["state", "location_state", "site_state"],
                    "year": ["year", "measurement_year", "date_year"],
                    "o3": ["ozone", "o3_concentration", "ozone_ppb"],
                    "so2": ["sulfur_dioxide", "so2_concentration", "so2_ppb"],
                    "pm": ["particulate_matter", "pm_concentration", "pm25"]
                }

                selected_cols = {}
                for key, alt_names in col_mapping.items():
                    for alt_name in alt_names:
                        if alt_name in df_sample.columns:
                            selected_cols[alt_name] = key
                            break  # Stop checking after finding the first match

                if selected_cols:
                    print(f"Using columns for {file}: {selected_cols}")

                    # Read full CSV with matched columns
                    df = pd.read_csv(file_path, usecols=selected_cols.keys(), low_memory=False, on_bad_lines='skip')

                    # Rename columns
                    df.rename(columns=selected_cols, inplace=True)

                    all_data.append(df)
                else:
                    print(f"Skipping {file}: No relevant columns found.")

            except pd.errors.ParserError as e:
                print(f"Skipping {file}: CSV Parsing Error - {e}")
            except Exception as e:
                print(f"Error processing {file}: {e}")

    # Merge all extracted data
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_csv(OUTPUT_CSV, index=False)
        print(f"Processed data saved to {OUTPUT_CSV}")
    else:
        print("No valid data extracted.")

# Run the function
process_air_quality_data(EXTRACTED_FOLDER)


Processing drydep_week_web.csv...
Columns found in drydep_week_web.csv: ['SITE_ID', 'YEAR', 'WEEK', 'DATEON', 'DATEOFF', 'VD_REPLACED_PCT', 'SO2_CONC', 'SO2_FLUX', 'SO4_CONC', 'SO4_FLUX', 'NO3_CONC', 'NO3_FLUX', 'HNO3_CONC', 'HNO3_FLUX', 'NH4_CONC', 'NH4_FLUX', 'CA_CONC', 'CA_FLUX', 'NA_CONC', 'NA_FLUX', 'MG_CONC', 'MG_FLUX', 'K_CONC', 'K_FLUX', 'CL_CONC', 'CL_FLUX']
Using columns for drydep_week_web.csv: {'year': 'year'}
Error processing drydep_week_web.csv: Usecols do not match columns, columns expected but not found: ['year']
Processing drydep_week_web_columninfo.csv...
Columns found in drydep_week_web_columninfo.csv: ['COLUMN_ID', 'COLUMN_NAME', 'UNIT', 'DESCRIPTION', 'DATA_TYPE', 'DATA_LENGTH', 'PRIMARY_KEY']
Skipping drydep_week_web_columninfo.csv: No relevant columns found.
Processing drydep_week_web_tableinfo.csv...
Columns found in drydep_week_web_tableinfo.csv: ['Table Name', 'DRYDEP_WEEK_WEB']
Skipping drydep_week_web_tableinfo.csv: No relevant columns found.
Processing hist

### Rate of Traumatic Brain Injury

**Source:** Health Data Government
**URL1:** http://healthdata.gov/
**URL2:** https://catalog.data.gov/dataset/rates-of-tbi-related-emergency-department-visits-hospitalizations-and-deaths-united-s-2001-36cef
**MIME Type:** XML
**Reasoning:** higher chance of visual/auditory hallucinations that can be easily misinterpreted as supernatural activity
**Features:**
1. TBI-Related Emergency Department (ED) Visit Rates
 - **Description:** This feature represents the rate of emergency department visits due to traumatic brain injuries per 100,000 individuals.
- **Hypothesis:** Higher rates of TBI-related ED visits in a region may correlate with increased reports of supernatural activity, as individuals with TBIs might experience symptoms such as hallucinations or altered perceptions, leading to misinterpretations of events as paranormal.
- **Additional Data Source:** Rates of TBI-related Emergency Department Visits, Hospitalizations, and Deaths - United States, 2001–2010.
- **Query to use:** Analyze whether regions with higher TBI-related ED visit rates have a corresponding increase in reported haunted locations.
2. TBI-Related Hospitalization Rates
- **Description:** This feature indicates the rate of hospitalizations due to traumatic brain injuries per 100,000 individuals.
- **Hypothesis:** Elevated hospitalization rates for TBIs might suggest a population with a higher prevalence of severe brain injuries, potentially leading to cognitive or sensory disturbances that could be perceived as supernatural experiences.
- **Additional Data Source:** Rates of TBI-related Emergency Department Visits, Hospitalizations, and Deaths - United States, 2001–2010.
- **Query to use:** Investigate the relationship between TBI-related hospitalization rates and the frequency of ghost sightings or haunted place reports in various regions.
3. TBI-Related Mortality Rates
- **Description:** This feature denotes the rate of deaths attributed to traumatic brain injuries per 100,000 individuals.
- **Hypothesis:** Regions with higher TBI-related mortality rates may have communities experiencing grief and trauma, potentially leading to an increased belief in or reporting of supernatural occurrences as a coping mechanism.
- **Additional Data Source:** Rates of TBI-related Emergency Department Visits, Hospitalizations, and Deaths - United States, 2001–2010.
- **Question to use:** Examine if there's a correlation between TBI-related death rates and the number of haunted places reported in those areas.

In [19]:
import os
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# URLs and file paths
XML_URL = "https://data.cdc.gov/api/views/45um-c62r/rows.xml?accessType=DOWNLOAD"
OUTPUT_CSV = "/work/Exports/brain injury data/tbi_data/tbi_data.csv"
DATA_FOLDER = "/work/Exports/brain injury data/tbi_data"

# Checks for data directory
os.makedirs(DATA_FOLDER, exist_ok=True)

# Download the XML data
response = requests.get(XML_URL)
xml_path = os.path.join(DATA_FOLDER, "tbi_data.xml")
with open(xml_path, "wb") as file:
    file.write(response.content)
print(f"Downloaded XML data to {xml_path}")

# Parse the XML data
tree = ET.parse(xml_path)
root = tree.getroot()

# Extract data dynamically
data = []
for entry in root.findall(".//row"):  
    record = {}
    for field in entry:
        tag = field.tag.split("}")[-1]  
        record[tag.lower()] = field.text 
    data.append(record)

# Convert to DataFrame
df = pd.DataFrame(data)

# Display detected column names
# print(f"Available columns in XML: {df.columns.tolist()}")

# **Map correct field names**
column_mapping = {
    "year": "year",
    "ed_visits": "tbi_ed_rate",
    "hospitalizations": "tbi_hospital_rate",
    "deaths": "tbi_death_rate"
}

# Filter only relevant columns and rename them
df = df[list(column_mapping.keys())].rename(columns=column_mapping)

# Convert numeric values
for col in ["tbi_ed_rate", "tbi_hospital_rate", "tbi_death_rate"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # Convert to float, set errors as NaN

# Save to CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"Data extraction complete! Saved to {OUTPUT_CSV}")


Downloaded XML data to /work/Exports/brain injury data/tbi_data/tbi_data.xml
Available columns in XML: ['row', 'year', 'ed_visits', 'hospitalizations', 'deaths', 'total']
Data extraction complete! Saved to /work/Exports/brain injury data/tbi_data/tbi_data.csv


### Mental Health Risks

**Source:** Mental Health America
**URL:** https://mhanational.org/mhamapping/mha-state-county-data
**MIME Type:** application/pdf
**Reasoning:** mental health illnesses like psychosis can similarly be the cause of paranormal experiences.
**Features:**
1. Prevalence of Depression
 - **Description:** This feature represents the percentage of individuals within a specific state or county who have screened positive for depression.
- **Hypothesis:** Higher prevalence of depression in a region may correlate with increased reports of paranormal activity, as depressive symptoms can include feelings of hopelessness and altered perceptions, potentially leading individuals to interpret experiences as supernatural.
- **Additional Data Source:** Mental Health America's County and State Data Map.
- **Query to use:** Analyze whether regions with higher depression prevalence have a corresponding increase in reported haunted locations.
2. Prevalence of Psychosis
- **Description:** This feature indicates the percentage of individuals within a specific state or county who have screened positive for psychosis.
- **Hypothesis:** Elevated rates of psychosis, which can involve hallucinations and delusions, might lead to a higher number of reported supernatural experiences, as individuals may misinterpret their perceptions as paranormal events.
- **Additional Data Source:** Mental Health America's County and State Data Map.
- **Query to use:** Investigate the relationship between psychosis prevalence and the frequency of ghost sightings or haunted place reports in various regions.
3. Prevalence of Post-Traumatic Stress Disorder (PTSD)
- **Description:** This feature denotes the percentage of individuals within a specific state or county who have screened positive for PTSD.
- **Hypothesis:** Regions with higher PTSD prevalence may have communities experiencing heightened anxiety and hypervigilance, potentially leading to an increased perception of paranormal occurrences as individuals may misinterpret environmental stimuli as threats.
- **Additional Data Source:** Mental Health America's County and State Data Map.
- **Question to use:** Examine if there's a correlation between PTSD prevalence rates and the number of haunted places reported in those areas.

In [1]:
#version #1

import os
import fitz  # PyMuPDF
import pandas as pd

# Directory where PDFs are stored
pdf_directory = "/work/Exports/mental_health_data/data files"

# List of PDF files
pdf_files = [
    "2021-mhcld-annual-detailed-tables.pdf",
    "2022-mhcld-annual-detailed-tables.pdf"
]

def extract_text_from_pdf(pdf_path):
    """Extract text from a given PDF file using PyMuPDF."""
    text_data = []
    doc = fitz.open(pdf_path)
    
    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text")  # Extract text
        text_data.append(text)
    
    return "\n".join(text_data)

def process_pdf_data(pdf_text):
    """Process extracted text into structured format (e.g., DataFrame)."""
    # Splitting by lines and extracting relevant sections
    lines = pdf_text.split("\n")
    
    # Assuming tabular data, filter meaningful lines (modify based on actual data structure)
    processed_data = [line.strip() for line in lines if line.strip()]
    
    return processed_data

# Extract and process data from each PDF
pdf_data = {}
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    extracted_text = extract_text_from_pdf(pdf_path)
    structured_data = process_pdf_data(extracted_text)
    
    # Store results
    pdf_data[pdf_file] = structured_data

# Convert to df
df = pd.DataFrame.from_dict(pdf_data, orient='index').transpose()

# Save as CSV
output_csv_path = "/work/Exports/mental_health_data/extracted_mh_data.csv"
df.to_csv(output_csv_path, index=False)

print(f"Extracted data saved to {output_csv_path}")


Extracted data saved to /work/Exports/mental_health_data/extracted_mh_data.csv


In [3]:
#Version 2
import os
import pdfplumber
import pandas as pd

# Path to the folder containing PDF files
PDF_FOLDER = "/work/Exports/mental_health_data/data files"
OUTPUT_CSV = "/work/Exports/mental_health_data/mental_health_extracted_tables.csv"

def extract_tables_from_pdfs(pdf_folder, max_pages=10):
    """Extracts tables from all PDFs in the given folder."""
    extracted_tables = []
    
    # List all PDF files in the folder
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

    if not pdf_files:
        print("No PDF files found in the directory.")
        return None

    for pdf_file in pdf_files:
        print(f"Processing {pdf_file}...")
        with pdfplumber.open(pdf_file) as pdf:
            num_pages = min(len(pdf.pages), max_pages)  # Limit to first few pages
            for page_num in range(num_pages):
                tables = pdf.pages[page_num].extract_tables()
                if tables:
                    extracted_tables.extend(tables)  # Collect all extracted tables

    return extracted_tables

def save_tables_to_csv(tables, output_path):
    """Converts extracted tables to CSV format and saves."""
    if not tables:
        print("No tables extracted.")
        return

    all_dataframes = []
    for table in tables:
        df = pd.DataFrame(table)  # Convert each table into a DataFrame
        all_dataframes.append(df)

    # Concatenate all extracted tables into one DataFrame
    final_df = pd.concat(all_dataframes, ignore_index=True)
    final_df.to_csv(output_path, index=False)
    print(f"Data successfully saved to {output_path}")

# Extract tables from PDFs
print("Extracting tables from PDFs...")
tables = extract_tables_from_pdfs(PDF_FOLDER, max_pages=10)

# Save extracted tables to CSV
save_tables_to_csv(tables, OUTPUT_CSV)


Extracting tables from PDFs...
Processing /work/Exports/mental_health_data/data files/2021-mhcld-annual-detailed-tables.pdf...
Processing /work/Exports/mental_health_data/data files/2022-mhcld-annual-detailed-tables.pdf...
Processing /work/Exports/mental_health_data/data files/Mental Health US-2010.pdf...
Data successfully saved to /work/Exports/mental_health_data/mental_health_extracted_tables.csv


In [10]:
# Version 3

import os
import pdfplumber
import pandas as pd
import re

# Constant Variables
PDF_FOLDER = "/work/Exports/mental_health_data/data files"
OUTPUT_DIR = "/work/Exports/mental_health_data/parsed_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def extract_tables_pdfplumber(pdf_path, start_page=11, max_pages=781):
    """Extract tables using pdfplumber, starting after TOC."""
    tables = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num in range(start_page - 1, min(len(pdf.pages), max_pages)):
                page = pdf.pages[page_num]
                extracted = page.extract_tables()
                if extracted:
                    tables.extend(extracted)
    except Exception as e:
        print(f"pdfplumber error for {pdf_path}: {e}")
    return tables

def normalize_table(table):
    """Normalize table structure."""
    if not table or not isinstance(table, list):
        return []
    try:
        max_cols = max(len(row) for row in table if isinstance(row, list) and row)
        return [row + [''] * (max_cols - len(row)) for row in table if isinstance(row, list) and row]
    except Exception as e:
        print(f"Normalization error: {e}")
        return []

def extract_features(table, year):
    """Extract State and features from table content."""
    df = pd.DataFrame(normalize_table(table))
    if df.empty:
        return None

    # Placeholder for feature extraction
    features = {"Year": year}
    state = None
    depression = None
    psychosis = None
    ptsd = None

    # Search for state
    for col in df.columns:
        if any(isinstance(cell, str) and re.search(r'(State|Region|Division)', cell, re.I) for cell in df[col]):
            state_col = df[col].dropna().iloc[0] if df[col].notna().any() else None
            if state_col and isinstance(state_col, str):
                state = state_col.split()[0] 

    # Search for features by keywords in headers or rows
    for col in df.columns:
        col_data = df[col].astype(str).str.lower()
        if 'depress' in ' '.join(col_data):
            depression = col_data.dropna().iloc[-1] if col_data.notna().any() else None 
        if 'psycho' in ' '.join(col_data) or 'schizo' in ' '.join(col_data):
            psychosis = col_data.dropna().iloc[-1] if col_data.notna().any() else None
        if 'trauma' in ' '.join(col_data) or 'ptsd' in ' '.join(col_data):
            ptsd = col_data.dropna().iloc[-1] if col_data.notna().any() else None

    features["State"] = state or "Unknown"
    features["Number_Depression"] = depression or "N/A"
    features["Number_Psychosis"] = psychosis or "N/A"
    features["Number_PTSD"] = ptsd or "N/A"

    # Add features as columns to the df
    for key, value in features.items():
        df[key] = value
    return df

def save_tables_to_csv(tables, year, output_path):
    """Convert tables to CSV with Year and features."""
    if not tables:
        print(f"No tables extracted for {year}.")
        return
    all_dataframes = []
    for table in tables:
        df = extract_features(table, year)
        if df is not None and not df.empty:
            all_dataframes.append(df)

    if all_dataframes:
        final_df = pd.concat(all_dataframes, ignore_index=True)
        final_df = final_df.replace('', pd.NA).dropna(how='all')
        final_df.to_csv(output_path, index=False)
        print(f"Data saved to {output_path} with {len(all_dataframes)} tables")
    else:
        print(f"No valid tables to save for {year}.")

def main():
    print("Extracting tables from PDFs...")
    for pdf_file in os.listdir(PDF_FOLDER):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(PDF_FOLDER, pdf_file)
            year = re.search(r'(\d{4})', pdf_file).group(1) if re.search(r'(\d{4})', pdf_file) else "Unknown"
            print(f"Processing {pdf_path}...")
            tables = extract_tables_pdfplumber(pdf_path)
            if tables:
                print(f"Found {len(tables)} tables with pdfplumber in {pdf_path}")
                output_path = os.path.join(OUTPUT_DIR, f"mental_health_extracted_tables_{year}.csv")
                save_tables_to_csv(tables, year, output_path)
            else:
                print(f"No tables found in {pdf_file}")

if __name__ == "__main__":
    main()

Extracting tables from PDFs...
Processing /work/Exports/mental_health_data/data files/2021-mhcld-annual-detailed-tables.pdf...
Found 349 tables with pdfplumber in /work/Exports/mental_health_data/data files/2021-mhcld-annual-detailed-tables.pdf
Data saved to /work/Exports/mental_health_data/parsed_output/mental_health_extracted_tables_2021.csv with 349 tables
Processing /work/Exports/mental_health_data/data files/2022-mhcld-annual-detailed-tables.pdf...
Found 351 tables with pdfplumber in /work/Exports/mental_health_data/data files/2022-mhcld-annual-detailed-tables.pdf
Data saved to /work/Exports/mental_health_data/parsed_output/mental_health_extracted_tables_2022.csv with 351 tables
Processing /work/Exports/mental_health_data/data files/Mental Health US-2010.pdf...
Found 238 tables with pdfplumber in /work/Exports/mental_health_data/data files/Mental Health US-2010.pdf
Data saved to /work/Exports/mental_health_data/parsed_output/mental_health_extracted_tables_2010.csv with 238 tables


### Integrating the Datasets

In [5]:
import pandas as pd

# Load the Haunted Places Dataset
haunted_df = pd.read_csv("/work/Datasets/haunted_places_evidence.tsv", sep="\t")

#Load the External Datasets
crime_df = pd.read_csv("/Exports/crime rates data/crime_rates_by_state.csv")
alcohol_df = pd.read_csv("/Exports/alcohol data/alcohol_consumption_by_state.csv")
daylight_df = pd.read_csv("/Exports/daylight duration data/daylight_duration_by_state.csv")

# Merge the datasets
# merged_df = pd.merge(crime_df, alcohol_df, on="State", how="outer")
# merged_df = pd.merge(merged_df, haunted_df, on="State", how="outer")

merged_df = haunted_df.merge(alcohol_df, on='state', how='left')
merged_df = merged_df.merge(daylight_df, on='state', how='left')
merged_df = merged_df.merge(crime_df, on='state', how='left')

merged_df.to_csv("/Exports/merged data/haunted_places_enriched.tsv", sep='\t', index=False)




FileNotFoundError: [Errno 2] No such file or directory: '/Exports/crime rates data/crime_rates_by_state.csv'

### Apply Apache Tika For Similarity Analysis

In [55]:
from tika import parser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Use Tika to Parse Text:
parsed = parser.from_file('/work/Exports/merged data/haunted_places_enriched.tsv')
content = parsed['content']

# Compute cosine similarity between descriptions
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(merged_df['description'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


FileNotFoundError: [Errno 2] No such file or directory: '/work/Exports/merged data/haunted_places_enriched.tsv'

### EXTRA CREDIT:

Add some new D3.js visualizations to Tika Similarity
a. Currently Tika Similarity only supports Dendrogram, Circle Packing, and combinations of those to view clusters, and relative similarities between
datasets
b. Download and install D3.js
i. Visit http://d3js.org/
ii. Review Mike Bostock’s Visual Gallery Wiki
iii. https://github.com/mbostock/d3/wiki/Tutorials
iv. Consider adding4. Assignment Setup
4.1 Group Formation
1. Feature related visualizations, e.g., time series, bar charts, plots
2. Add functionality in a generic way that is not specific to your dataset
3. See gallery here: https://github.com/d3/d3/wiki/Gallery
4. Contributions will be reviewed as Pull Requests in a first come, first serve basis (check existing PRs and make sure
you aren’t duplicating what some other group has done)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9721ae8a-6e85-461c-a6dc-c5ce91a700f2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>